# 5章　カテゴリデータの取り扱い 
## レシピ5.1　名義カテゴリ特徴量の数値化 

In [ ]:
# ライブラリをロード
import numpy as np
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer

# 特徴量を作成
feature = np.array([["Texas"],
                    ["California"],
                    ["Texas"],
                    ["Delaware"],
                    ["Texas"]])

# ワンホットエンコーダを作成
one_hot = LabelBinarizer()

# 特徴量をワンホットエンコード
one_hot.fit_transform(feature)

In [ ]:
# 特徴量クラスを表示
one_hot.classes_

In [ ]:
# ワンホットエンコードされた特徴量を逆変換
one_hot.inverse_transform(one_hot.transform(feature))

In [ ]:

# ライブラリをロード
import pandas as pd

# 特徴量からダミー変数を生成
pd.get_dummies(feature[:,0])

In [ ]:

# 複数クラス特徴量を作成
multiclass_feature = [("Texas", "Florida"),
                      ("California", "Alabama"),
                      ("Texas", "Florida"),
                      ("Delaware", "Florida"),
                      ("Texas", "Alabama")]

# 複数クラス用ワンホットエンコーダを作成
one_hot_multiclass = MultiLabelBinarizer()

# 複数クラス特徴量をワンホットエンコード
one_hot_multiclass.fit_transform(multiclass_feature)

In [ ]:
# クラスを表示
one_hot_multiclass.classes_

## レシピ5.2　順序カテゴリ特徴量の数値化 


In [ ]:
# ライブラリをロード
import pandas as pd

# 特徴量を作成
dataframe = pd.DataFrame({"Score": ["Low", "Low", "Medium", "Medium", "High"]})

# マップを作成
scale_mapper = {"Low":1,
                "Medium":2,
                "High":3}

# 特徴量の値をマップを使って置換
dataframe["Score"].replace(scale_mapper)


In [ ]:
dataframe = pd.DataFrame({"Score": ["Low",
                                    "Low",
                                    "Medium",
                                    "Medium",
                                    "High",
                                    "Barely More Than Medium"]})
scale_mapper = {"Low":1,
                "Medium":2,
                "Barely More Than Medium": 3,
                "High":4}

dataframe["Score"].replace(scale_mapper)

In [ ]:
scale_mapper = {"Low":1,
                "Medium":2,
                "Barely More Than Medium": 2.1,
                "High":3}

dataframe["Score"].replace(scale_mapper)

## レシピ5.3　特徴量辞書の数値化 


In [ ]:
# ライブラリをロード
from sklearn.feature_extraction import DictVectorizer

# 辞書を作成
data_dict = [{"Red": 2, "Blue": 4},
             {"Red": 4, "Blue": 3},
             {"Red": 1, "Yellow": 2},
             {"Red": 2, "Yellow": 2}]

# 辞書ベクトル変換器を作成
dictvectorizer = DictVectorizer(sparse=False)

# 辞書を特徴量行列に変換
features = dictvectorizer.fit_transform(data_dict)

# 特徴量行列を表示
features

In [ ]:
# 特徴量の名前を取得
feature_names = dictvectorizer.get_feature_names_out()

# 特徴量の名前を表示
feature_names

In [ ]:
# ライブラリをロード
import pandas as pd

# 特徴量からDataFrameを作成
pd.DataFrame(features, columns=feature_names)

In [ ]:
# 4つの文書に対する単語カウント辞書を作成
doc_1_word_count = {"Red": 2, "Blue": 4}
doc_2_word_count = {"Red": 4, "Blue": 3}
doc_3_word_count = {"Red": 1, "Yellow": 2}
doc_4_word_count = {"Red": 2, "Yellow": 2}

# リストを作成
doc_word_counts = [doc_1_word_count,
                   doc_2_word_count,
                   doc_3_word_count,
                   doc_4_word_count]

# 単語カウント辞書のリストを特徴量行列に変換
dictvectorizer.fit_transform(doc_word_counts)

## レシピ5.4　欠損クラス値の補完 


In [ ]:
# ライブラリをロード
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

# カテゴリ特徴量を持つ特徴量行列を作成
X = np.array([[0, 2.10, 1.45],
              [1, 1.18, 1.33],
              [0, 1.22, 1.27],
              [1, -0.21, -1.19]])

# カテゴリ特徴量に欠損値を持つ特徴量行列を作成
X_with_nan = np.array([[np.nan, 0.87, 1.31],
                       [np.nan, -0.67, -0.22]])

# KNNクラス分類器を訓練
clf = KNeighborsClassifier(3, weights='distance')
trained_model = clf.fit(X[:,1:], X[:,0])

# 欠損値のクラスを予測
imputed_values = trained_model.predict(X_with_nan[:,1:])

# 予測されたクラス値と他の特徴量を結合
X_with_imputed = np.hstack((imputed_values.reshape(-1,1), X_with_nan[:,1:]))

# 2つの特徴量行列を結合
np.vstack((X_with_imputed, X))

In [ ]:
# ライブラリをロード
from sklearn.impute import SimpleImputer

# 2つの特徴量行列を結合
X_complete = np.vstack((X_with_nan, X))

imputer = SimpleImputer(strategy='most_frequent')

imputer.fit_transform(X_complete)

## レシピ5.5　不均等なクラスの取り扱い 


In [ ]:
# ライブラリをロード
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris

# irisデータセットをロード
iris = load_iris()

# 特徴量行列を作成
features = iris.data

# ターゲットベクトルを作成
target = iris.target

# 最初の40の観測値を削除
features = features[40:,:]
target = target[40:]

# クラス0であるかどうかを示す2値ターゲットベクトルを作成
target = np.where((target == 0), 0, 1)

# バランスの崩れたターゲットベクトルを表示
target

In [ ]:
# 重みを作成
weights = {0: .9, 1: 0.1}

# ランダムフォレストクラス分類器を、重みを指定して作成
RandomForestClassifier(class_weight=weights)

In [ ]:
# ランダムフォレストクラス分類器を、重みをbalancedに指定して作成
RandomForestClassifier(class_weight="balanced")

In [ ]:
# それぞれのクラスの観測値のインデックスを取得
i_class0 = np.where(target == 0)[0]
i_class1 = np.where(target == 1)[0]

# それぞれのクラスの観測値数を計算
n_class0 = len(i_class0)
n_class1 = len(i_class1)

# クラス0のそれぞれの観測値に対して、ランダムにクラス1から非復元抽出
i_class1_downsampled = np.random.choice(i_class1, size=n_class0, replace=False)

# クラス0のターゲットベクトルと、
# ダウンサンプリングしたクラス1のターゲットベクトルを結合
np.hstack((target[i_class0], target[i_class1_downsampled]))

In [ ]:
# クラス0の特徴量行列と、ダウンサンプリングしたクラス1の特徴量行列を結合
np.vstack((features[i_class0,:], features[i_class1_downsampled,:]))[0:5]

In [ ]:
# クラス1のそれぞれの観測値に対して、ランダムにクラス0から復元抽出
i_class0_upsampled = np.random.choice(i_class0, size=n_class1, replace=True)

# クラス0のアップサンプリングされたターゲットベクトルとクラス1のターゲットベクトルを結合
np.concatenate((target[i_class0_upsampled], target[i_class1]))

In [ ]:
# クラス0をアップサンプリングした特徴量行列と、クラス1の特徴量行列を結合
np.vstack((features[i_class0_upsampled,:], features[i_class1,:]))[0:5]